In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import math


In [2]:
def make_signal(w,theta,n):
    """
    Assumes normalized amplitude
    """
    t = np.arange(n)
    signal = np.exp(1j*(w*t + theta))
    return signal

def make_noise(sigma2,n):
    noise_scaling = np.sqrt(sigma2/2)
    # noise is complex valued
    noise  = noise_scaling*np.random.randn(n) + 1j*noise_scaling*np.random.randn(n)
    return noise

def make_noisy_signal(w,theta,SNRdb,n):
    sigma2 = get_sigma2_from_snrdb(SNRdb)
    signal = make_signal(w,theta,n)
    noise  = make_noise(sigma2,n)
    return signal + noise

# N = divisor of w0
# m = num samples
def make_batch_noisy(batch_size, SNRdb, N, m, binary=False):
    signals, freqs = [], []
    for i in range(batch_size):
        freq = np.random.randint(0, N)
        w = (2 * np.pi * freq / N) % (2 * np.pi)
        sig = make_noisy_signal(w, 0, SNRdb, m)
        signals.append(sig)
        freqs.append(freq)
    if binary:
        return signals, make_binary(freqs, N), one_hot(N, batch_size, freqs)
    return signals, one_hot(N, batch_size, freqs)

# N = divisor of w0
# m = num samples
def make_batch_noisy_lohi(batch_size, SNRdb, N, m):
    freqs = []
    freqs.append(np.random.randint(0, N))
    test_signals, test_freqs = make_noisy_lohi(SNRdB, N, m, freqs[-1])
    for i in range(1, batch_size):
        freqs.append(np.random.randint(0, N))
        a, b = make_noisy_lohi(SNRdB, N, m, freqs[-1])
        test_signals.extend(a)
        test_freqs.extend(b)
    return test_signals, test_freqs, freqs

def make_noisy_lohi(SNRdb, N, m, freq):
    signals, vals = [], []
    steps = int(np.log2(N))
    w = (2 * np.pi * freq / N) % (2 * np.pi)
    sig = make_noisy_signal(w, 0, SNRdb, m * (2**steps))
    for i in range(int(np.log2(N))):
        signals.append([sig[a * (2**i)] for a in range(m)])
        if (freq * (2**i)) % (N) < N / 2:
            vals.append([1])
        else:
            vals.append([0])
    return signals, vals
        

def make_batch_singleton(batch_size, SNRdb, N, m, default=-1): # 0 = zero, 1 = single, 2 = multi
    signals, freqs = [], []
    sigma2 = get_sigma2_from_snrdb(SNRdB)
    for i in range(batch_size):
        val = np.random.poisson(0.79)
        if default >= 0:
            val = default
        if val == 0:
            signals.append(make_noise(0, m))
            freqs.append([1, 0, 0])
        if val == 1:
            signals.append(make_noisy_signal(2 * np.pi * np.random.randint(0, N) / N, 0, SNRdB, m))
            freqs.append([0, 1, 0])
        if val >= 2:
            signal = make_signal(2 * np.pi * np.random.randint(0, N) / N, 0, m)
            for i in range(val - 1):
                signal += make_signal(2 * np.pi * np.random.randint(0, N) / N, 0, m)
            signals.append(signal + make_noise(sigma2, m))
            freqs.append([0, 0, 1])
    return signals, freqs

def get_sigma2_from_snrdb(SNR_db):
    return 10**(-SNR_db/10)

def kay_weights(N):
    scaling = (3.0/2)*N/(N**2 - 1)
    
    w = [1 - ((i - (N/2 - 1))/(N/2))**2 for i in range(N-1)]
    
    return scaling*np.array(w)

def kays_method(my_signal):
    N = len(my_signal)
    w = kay_weights(N)
    
    angle_diff = np.angle(np.conj(my_signal[0:-1])*my_signal[1:])
    need_to_shift = np.any(angle_diff < -np.pi/2)
    if need_to_shift:    
        neg_idx = angle_diff < 0
        angle_diff[neg_idx] += np.pi*2
    
    return w.dot(angle_diff)

def kays_singleton_accuracy(test_signals, test_freqs, N):
    diffs = [s - make_signal(kays_method(s), 0, N) for s in test_signals]
    thresh, single_acc, other_acc, best_thresh = 0.0, 0, 0, 0
    best = 0
    for i in range(150):
        vals = [(sum(np.absolute(s)) / N) < thresh for s in diffs]
        corr = [1 for i in range(len(test_freqs)) if (test_freqs[i] == [0, 1, 0] and vals[i] == 1) or ((test_freqs[i] != [0, 1, 0] and vals[i] == 0))]
        corr = sum(corr)
        #single = sum([vals[d] for d in range(len(vals)) if test_freqs[d] == [0, 1, 0]]) / len([vals[d] for d in range(len(vals)) if test_freqs[d] == [0, 1, 0]])
        #other = sum([not vals[d] for d in range(len(vals)) if test_freqs[d] != [0, 1, 0]]) / len([vals[d] for d in range(len(vals)) if test_freqs[d] != [0, 1, 0]])        
        #if single*2 + other > single_acc*2 + other_acc and single > 0.2 and other > 0.2:
        #    single_acc = single
        #    other_acc = other
        #    best_thresh = thresh
        if corr > best:
            best = corr
            best_thresh = thresh
        thresh += 0.05
    print('thresh: ', best_thresh)
    return best / len(test_signals)

def test_kays(signals, freqs, N):
    count = 0
    for sig, freq in zip(signals, freqs):
        res = kays_method(sig)
        res = round(res * N / (2 * np.pi))
        if np.argmax(freq) == res:
            count += 1
    return count / len(signals)

def test_mle(signals, freqs, N, m):
    count = 0
    for sig, freq in zip(signals, freqs):
        cleans = [make_signal(np.pi * 2 * w / N, 0, m) for w in range(N)]
        dots = [np.absolute(np.vdot(sig, clean)) for clean in cleans]
        if np.argmax(dots) == np.argmax(freq):
            count += 1
    return count / len(signals)
    
def make_binary(freqs, N):
    w = math.ceil(np.log2(N))
    return [[int(a) for a in list(np.binary_repr(f, width=w))] for f in freqs] 

def binary_to_int(binary_string):
    return tf.reduce_sum(
    tf.cast(tf.reverse(tensor=binary_string, axis=[0]), dtype=tf.int64)
    * 2 ** tf.range(tf.cast(tf.size(binary_string), dtype=tf.int64)))
    '''y = 0
    for i,j in enumerate(x):
        y += j<<i
    return y'''

def hamming(pred, act):
    return np.count_nonzero(pred != act)

def one_hot(N, batch_size, freqs):
    freqs_one_hot = np.zeros((batch_size, N))
    freqs_one_hot[np.arange(batch_size), freqs] = 1
    return freqs_one_hot

def test_noisy_mle(N, m, signals, freqs):
    count = 0  
    '''imag_signals = []
    for index in range(len(signals)):
        sig = signals[index]
        imag_sig = [(sig[i] + 1j*sig[i+1]) for i in np.arange(len(sig), step=2)]
        imag_signals.append(imag_sig)'''
    cleans = [make_signal(2*np.pi*i/N, 0, m) for i in range(N)]
                     
    for index in range(len(signals)):
        dots = [np.absolute(np.vdot(signals[index], cleans[i])) for i in range(N)]
        if np.argmax(freqs[index]) == np.argmax(dots):
            #print(np.argmax(dots))
            count += 1
    return count / len(freqs)

def bit_to_freq(bits, N):
    possible = [i for i in range(N)]
    for b in bits:
        if b[0]:
            possible = possible[:len(possible)//2]
        else:
            possible = possible[len(possible)//2:]
    return possible[0]

In [3]:
# div and conquer freq detect

N = 1024 #8192 
SNRdB = -2
layer = 6
#m = 50

log = int(np.log2(N))

# Parameters
learning_rate = 0.005
num_iter = 80000
batch_size = log * 20


# Network Parameters
num_classes = 1

ms = [20, 30, 50, 80, 120, 200]

bin_accs = []
freq_accs = []


for m in ms:

    t_bins = []
    t_freqs = []
    for trial in range(4):
        # tf Graph input
        X = tf.placeholder("float", [None, m, 2])
        Y = tf.placeholder("float", [None, num_classes])

        # Store layers weight & bias
        weights = {i: tf.Variable(tf.random_normal([3, 2, 2])) for i in range(1, layer+1)}
        weights[0] = tf.Variable(tf.random_normal([5, 2, 2]))
        weights['out'] = tf.Variable(tf.random_normal([(m-4-(2*layer))*2, num_classes]))
        biases = {i: tf.Variable(tf.random_normal([2])) for i in range(layer+1)}
        biases['out'] = tf.Variable(tf.random_normal([num_classes]))


        def neural_net(x):
            layer_1 = tf.add(tf.nn.conv1d(x, weights[0], 1, 'VALID'), biases[0])
            hidden_1 = tf.nn.relu(layer_1)
            for i in range(1, layer+1):
                layer_1 = tf.add(tf.nn.conv1d(hidden_1, weights[i], 1, 'VALID'), biases[i])
                hidden_1 = tf.nn.relu(layer_1)
            hidden_3 = tf.reshape(hidden_1, [batch_size, -1])
            out_layer = tf.matmul(hidden_3, weights['out']) + biases['out']
            return out_layer


        '''weights = {
            'h1': tf.Variable(tf.random_normal([5, 2, 2])), # filtersize, in channels, outchannels
            'out': tf.Variable(tf.random_normal([(m-4-2-2)*2, num_classes])),
            'h2': tf.Variable(tf.random_normal([3, 2, 2])),
            'h3': tf.Variable(tf.random_normal([3, 2, 2]))
        }
        biases = {
            'b1': tf.Variable(tf.random_normal([2])),
            'out': tf.Variable(tf.random_normal([num_classes])),
            'b2': tf.Variable(tf.random_normal([2])),
            'b3': tf.Variable(tf.random_normal([2]))
        }

        def neural_net(x):
            layer_1 = tf.add(tf.nn.conv1d(x, weights['h1'], 1, 'VALID'), biases['b1'])
            hidden_1 = tf.nn.relu(layer_1)
            layer_2 = tf.add(tf.nn.conv1d(hidden_1, weights['h2'], 1, 'VALID'), biases['b2'])
            hidden_2 = tf.nn.relu(layer_2)
            layer_3 = tf.add(tf.nn.conv1d(hidden_2, weights['h3'], 1, 'VALID'), biases['b3'])
            hidden_3 = tf.nn.relu(layer_3)
            hidden_3 = tf.reshape(hidden_3, [batch_size, -1])
            out_layer = tf.matmul(hidden_3, weights['out']) + biases['out']
            return out_layer
        '''
        test_dict = {}
        for i in range(10):
            test_signals, test_freqs, freqs = make_batch_noisy_lohi(batch_size // log, SNRdB, N, m)
            test_signals_pair = np.zeros((batch_size, m, 2))
            test_signals_pair[:, :, 0] = np.real(test_signals)
            test_signals_pair[:, :, 1] = np.imag(test_signals)
            test_dict[i] = (test_signals_pair, test_freqs, freqs)


        training_size = 3999
        dict = {}
        for i in range(training_size):
            batch_x, batch_y, batch_freqs = make_batch_noisy_lohi(batch_size // log, SNRdB - (i % 3), N, m)
            batch_x_pair = np.zeros((batch_size, m, 2))
            batch_x_pair[:, :, 0] = np.real(batch_x)
            batch_x_pair[:, :, 1] = np.imag(batch_x)
            dict[i] = (batch_x_pair, batch_y)



        # Construct model
        logits = neural_net(X)
        prediction = tf.nn.sigmoid(logits)
        losses, accuracies = [], []

        # Define loss and optimizer
        loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
            logits=logits, labels=Y))  

        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        train_op = optimizer.minimize(loss_op)

        # Evaluate model
        pred_class = tf.greater(prediction, 0.5)
        correct_pred = tf.equal(pred_class, tf.equal(Y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

        # Initialize the variables (i.e. assign their default value)
        init = tf.global_variables_initializer()

        # Start training
        with tf.Session() as sess:

            # Run the initializer
            sess.run(init)
            print("Training Started")

            for step in range(1, num_iter + 1):
                batch_x_pair, batch_y = dict[step % training_size]

                # Run optimization op (backprop)
                sess.run(train_op, feed_dict={X: batch_x_pair, Y: batch_y})
                if step % 500 == 0:
                    # Calculate batch loss and accuracy
                    loss, acc, pred = sess.run([loss_op, accuracy, prediction], feed_dict={X: batch_x_pair,
                                                                         Y: batch_y})

                    accuracies.append(acc)
                    losses.append(loss)
                    print("Iter " + str(step) + ", Minibatch Loss= " + \
                          "{:.4f}".format(loss) + ", Training Accuracy= " + \
                          "{:.3f}".format(acc))
            print("Training Finished")
            preds = [sess.run(prediction, feed_dict={X: test_dict[i][0], Y: test_dict[i][1]}) for i in range(len(test_dict))]  
            nn_acc = [sess.run(accuracy, feed_dict={X: test_dict[i][0], Y: test_dict[i][1]}) for i in range(len(test_dict))]   
            print(nn_acc)

        t_bins.append(np.median(preds))
        preds = np.round(preds)
        corr = []
        for a in range(len(test_dict)):
            fs = []
            for k in range(len(preds[a]) // log):
                    fs.append(bit_to_freq(preds[a][k * log : (k+1) * log], N))
            corr.extend([fs[i] == test_dict[a][2][i] % N for i in range(len(fs))])
        t_freqs.append(np.sum(corr) / (len(fs) * len(test_dict)))
    bin_accs.append(max(t_bins))
    freq_accs.append(max(t_freqs))
    print(bin_accs[-1])
    print(freq_accs[-1])
    
np.save('./data/divide_conquer/ms', ms)   
np.save('./data/divide_conquer/bin_accs', bin_accs)
np.save('./data/divide_conquer/freq_accs', freq_accs)




Training Started
Iter 500, Minibatch Loss= 0.7050, Training Accuracy= 0.545
Iter 1000, Minibatch Loss= 0.5607, Training Accuracy= 0.750
Iter 1500, Minibatch Loss= 0.4429, Training Accuracy= 0.805
Iter 2000, Minibatch Loss= 0.3807, Training Accuracy= 0.820
Iter 2500, Minibatch Loss= 0.3464, Training Accuracy= 0.880
Iter 3000, Minibatch Loss= 0.3483, Training Accuracy= 0.845
Iter 3500, Minibatch Loss= 0.3480, Training Accuracy= 0.870
Iter 4000, Minibatch Loss= 0.2366, Training Accuracy= 0.900
Iter 4500, Minibatch Loss= 0.2180, Training Accuracy= 0.925
Iter 5000, Minibatch Loss= 0.2096, Training Accuracy= 0.905
Iter 5500, Minibatch Loss= 0.2677, Training Accuracy= 0.900
Iter 6000, Minibatch Loss= 0.2874, Training Accuracy= 0.895
Iter 6500, Minibatch Loss= 0.1848, Training Accuracy= 0.925
Iter 7000, Minibatch Loss= 0.2845, Training Accuracy= 0.885
Iter 7500, Minibatch Loss= 0.1698, Training Accuracy= 0.930
Iter 8000, Minibatch Loss= 0.1933, Training Accuracy= 0.935
Iter 8500, Minibatch Los

Iter 68000, Minibatch Loss= 0.2181, Training Accuracy= 0.915
Iter 68500, Minibatch Loss= 0.1665, Training Accuracy= 0.930
Iter 69000, Minibatch Loss= 0.1389, Training Accuracy= 0.945
Iter 69500, Minibatch Loss= 0.1160, Training Accuracy= 0.950
Iter 70000, Minibatch Loss= 0.1448, Training Accuracy= 0.945
Iter 70500, Minibatch Loss= 0.1475, Training Accuracy= 0.940
Iter 71000, Minibatch Loss= 0.2317, Training Accuracy= 0.900
Iter 71500, Minibatch Loss= 0.1692, Training Accuracy= 0.925
Iter 72000, Minibatch Loss= 0.2589, Training Accuracy= 0.895
Iter 72500, Minibatch Loss= 0.2316, Training Accuracy= 0.915
Iter 73000, Minibatch Loss= 0.2443, Training Accuracy= 0.900
Iter 73500, Minibatch Loss= 0.2020, Training Accuracy= 0.915
Iter 74000, Minibatch Loss= 0.1632, Training Accuracy= 0.935
Iter 74500, Minibatch Loss= 0.2092, Training Accuracy= 0.930
Iter 75000, Minibatch Loss= 0.1734, Training Accuracy= 0.930
Iter 75500, Minibatch Loss= 0.2374, Training Accuracy= 0.905
Iter 76000, Minibatch Lo

Iter 54500, Minibatch Loss= 0.2827, Training Accuracy= 0.890
Iter 55000, Minibatch Loss= 0.2613, Training Accuracy= 0.900
Iter 55500, Minibatch Loss= 0.2726, Training Accuracy= 0.895
Iter 56000, Minibatch Loss= 0.1575, Training Accuracy= 0.925
Iter 56500, Minibatch Loss= 0.1955, Training Accuracy= 0.935
Iter 57000, Minibatch Loss= 0.2041, Training Accuracy= 0.905
Iter 57500, Minibatch Loss= 0.2216, Training Accuracy= 0.920
Iter 58000, Minibatch Loss= 0.2185, Training Accuracy= 0.930
Iter 58500, Minibatch Loss= 0.1335, Training Accuracy= 0.955
Iter 59000, Minibatch Loss= 0.1975, Training Accuracy= 0.925
Iter 59500, Minibatch Loss= 0.2009, Training Accuracy= 0.910
Iter 60000, Minibatch Loss= 0.1857, Training Accuracy= 0.940
Iter 60500, Minibatch Loss= 0.1658, Training Accuracy= 0.930
Iter 61000, Minibatch Loss= 0.1843, Training Accuracy= 0.930
Iter 61500, Minibatch Loss= 0.1867, Training Accuracy= 0.925
Iter 62000, Minibatch Loss= 0.2989, Training Accuracy= 0.900
Iter 62500, Minibatch Lo

Iter 41000, Minibatch Loss= 0.6946, Training Accuracy= 0.470
Iter 41500, Minibatch Loss= 0.6946, Training Accuracy= 0.450
Iter 42000, Minibatch Loss= 0.6932, Training Accuracy= 0.490
Iter 42500, Minibatch Loss= 0.6932, Training Accuracy= 0.500
Iter 43000, Minibatch Loss= 0.6929, Training Accuracy= 0.535
Iter 43500, Minibatch Loss= 0.6936, Training Accuracy= 0.435
Iter 44000, Minibatch Loss= 0.6931, Training Accuracy= 0.535
Iter 44500, Minibatch Loss= 0.6931, Training Accuracy= 0.505
Iter 45000, Minibatch Loss= 0.6940, Training Accuracy= 0.485
Iter 45500, Minibatch Loss= 0.6934, Training Accuracy= 0.495
Iter 46000, Minibatch Loss= 0.6931, Training Accuracy= 0.495
Iter 46500, Minibatch Loss= 0.6941, Training Accuracy= 0.445
Iter 47000, Minibatch Loss= 0.6933, Training Accuracy= 0.475
Iter 47500, Minibatch Loss= 0.6929, Training Accuracy= 0.535
Iter 48000, Minibatch Loss= 0.6932, Training Accuracy= 0.490
Iter 48500, Minibatch Loss= 0.6934, Training Accuracy= 0.470
Iter 49000, Minibatch Lo

Iter 27500, Minibatch Loss= 0.1929, Training Accuracy= 0.915
Iter 28000, Minibatch Loss= 0.1897, Training Accuracy= 0.925
Iter 28500, Minibatch Loss= 0.1664, Training Accuracy= 0.955
Iter 29000, Minibatch Loss= 0.2425, Training Accuracy= 0.900
Iter 29500, Minibatch Loss= 0.1514, Training Accuracy= 0.935
Iter 30000, Minibatch Loss= 0.1940, Training Accuracy= 0.945
Iter 30500, Minibatch Loss= 0.2670, Training Accuracy= 0.890
Iter 31000, Minibatch Loss= 0.1759, Training Accuracy= 0.930
Iter 31500, Minibatch Loss= 0.1826, Training Accuracy= 0.925
Iter 32000, Minibatch Loss= 0.2614, Training Accuracy= 0.900
Iter 32500, Minibatch Loss= 0.1912, Training Accuracy= 0.935
Iter 33000, Minibatch Loss= 0.1662, Training Accuracy= 0.925
Iter 33500, Minibatch Loss= 0.2277, Training Accuracy= 0.910
Iter 34000, Minibatch Loss= 0.1703, Training Accuracy= 0.925
Iter 34500, Minibatch Loss= 0.1856, Training Accuracy= 0.920
Iter 35000, Minibatch Loss= 0.1672, Training Accuracy= 0.930
Iter 35500, Minibatch Lo

...

Iter 2500, Minibatch Loss= 0.6927, Training Accuracy= 0.505
Iter 3000, Minibatch Loss= 0.6925, Training Accuracy= 0.505
Iter 3500, Minibatch Loss= 0.6944, Training Accuracy= 0.475
Iter 4000, Minibatch Loss= 0.6932, Training Accuracy= 0.480
Iter 4500, Minibatch Loss= 0.6935, Training Accuracy= 0.490
Iter 5000, Minibatch Loss= 0.6932, Training Accuracy= 0.500
Iter 5500, Minibatch Loss= 0.6934, Training Accuracy= 0.475
Iter 6000, Minibatch Loss= 0.6932, Training Accuracy= 0.500
Iter 6500, Minibatch Loss= 0.6941, Training Accuracy= 0.435
Iter 7000, Minibatch Loss= 0.6917, Training Accuracy= 0.545
Iter 7500, Minibatch Loss= 0.6931, Training Accuracy= 0.505
Iter 8000, Minibatch Loss= 0.6931, Training Accuracy= 0.505
Iter 8500, Minibatch Loss= 0.6922, Training Accuracy= 0.540
Iter 9000, Minibatch Loss= 0.6929, Training Accuracy= 0.560
Iter 9500, Minibatch Loss= 0.6932, Training Accuracy= 0.490
Iter 10000, Minibatch Loss= 0.6934, Training Accuracy= 0.490
Iter 10500, Minibatch Loss= 0.6936, Tra

Iter 70000, Minibatch Loss= 0.6934, Training Accuracy= 0.490
Iter 70500, Minibatch Loss= 0.6934, Training Accuracy= 0.430
Iter 71000, Minibatch Loss= 0.6937, Training Accuracy= 0.470
Iter 71500, Minibatch Loss= 0.6940, Training Accuracy= 0.485
Iter 72000, Minibatch Loss= 0.6928, Training Accuracy= 0.515
Iter 72500, Minibatch Loss= 0.6942, Training Accuracy= 0.450
Iter 73000, Minibatch Loss= 0.6950, Training Accuracy= 0.415
Iter 73500, Minibatch Loss= 0.6932, Training Accuracy= 0.465
Iter 74000, Minibatch Loss= 0.6919, Training Accuracy= 0.565
Iter 74500, Minibatch Loss= 0.6932, Training Accuracy= 0.495
Iter 75000, Minibatch Loss= 0.6933, Training Accuracy= 0.495
Iter 75500, Minibatch Loss= 0.6914, Training Accuracy= 0.545
Iter 76000, Minibatch Loss= 0.6946, Training Accuracy= 0.460
Iter 76500, Minibatch Loss= 0.6931, Training Accuracy= 0.505
Iter 77000, Minibatch Loss= 0.6926, Training Accuracy= 0.530
Iter 77500, Minibatch Loss= 0.6932, Training Accuracy= 0.465
Iter 78000, Minibatch Lo

Iter 57000, Minibatch Loss= 0.6929, Training Accuracy= 0.570
Iter 57500, Minibatch Loss= 0.6933, Training Accuracy= 0.495
Iter 58000, Minibatch Loss= 0.6917, Training Accuracy= 0.575
Iter 58500, Minibatch Loss= 0.6931, Training Accuracy= 0.500
Iter 59000, Minibatch Loss= 0.6928, Training Accuracy= 0.540
Iter 59500, Minibatch Loss= 0.6932, Training Accuracy= 0.500
Iter 60000, Minibatch Loss= 0.6936, Training Accuracy= 0.445
Iter 60500, Minibatch Loss= 0.6922, Training Accuracy= 0.550
Iter 61000, Minibatch Loss= 0.6932, Training Accuracy= 0.480
Iter 61500, Minibatch Loss= 0.6929, Training Accuracy= 0.515
Iter 62000, Minibatch Loss= 0.6936, Training Accuracy= 0.480
Iter 62500, Minibatch Loss= 0.6932, Training Accuracy= 0.480
Iter 63000, Minibatch Loss= 0.6929, Training Accuracy= 0.530
Iter 63500, Minibatch Loss= 0.6940, Training Accuracy= 0.450
Iter 64000, Minibatch Loss= 0.6929, Training Accuracy= 0.545
Iter 64500, Minibatch Loss= 0.6928, Training Accuracy= 0.520
Iter 65000, Minibatch Lo

Iter 43500, Minibatch Loss= 0.6923, Training Accuracy= 0.550
Iter 44000, Minibatch Loss= 0.6931, Training Accuracy= 0.520
Iter 44500, Minibatch Loss= 0.6932, Training Accuracy= 0.485
Iter 45000, Minibatch Loss= 0.6930, Training Accuracy= 0.510
Iter 45500, Minibatch Loss= 0.6931, Training Accuracy= 0.500
Iter 46000, Minibatch Loss= 0.6925, Training Accuracy= 0.530
Iter 46500, Minibatch Loss= 0.6938, Training Accuracy= 0.470
Iter 47000, Minibatch Loss= 0.6931, Training Accuracy= 0.515
Iter 47500, Minibatch Loss= 0.6931, Training Accuracy= 0.505
Iter 48000, Minibatch Loss= 0.6932, Training Accuracy= 0.490
Iter 48500, Minibatch Loss= 0.6932, Training Accuracy= 0.460
Iter 49000, Minibatch Loss= 0.6944, Training Accuracy= 0.455
Iter 49500, Minibatch Loss= 0.6930, Training Accuracy= 0.530
Iter 50000, Minibatch Loss= 0.6931, Training Accuracy= 0.505
Iter 50500, Minibatch Loss= 0.6936, Training Accuracy= 0.480
Iter 51000, Minibatch Loss= 0.6931, Training Accuracy= 0.495
Iter 51500, Minibatch Lo

Iter 30000, Minibatch Loss= 0.6935, Training Accuracy= 0.495
Iter 30500, Minibatch Loss= 0.6931, Training Accuracy= 0.505
Iter 31000, Minibatch Loss= 0.6931, Training Accuracy= 0.505
Iter 31500, Minibatch Loss= 0.6933, Training Accuracy= 0.470
Iter 32000, Minibatch Loss= 0.6942, Training Accuracy= 0.460
Iter 32500, Minibatch Loss= 0.6932, Training Accuracy= 0.505
Iter 33000, Minibatch Loss= 0.6930, Training Accuracy= 0.510
Iter 33500, Minibatch Loss= 0.6920, Training Accuracy= 0.540
Iter 34000, Minibatch Loss= 0.6945, Training Accuracy= 0.465
Iter 34500, Minibatch Loss= 0.6931, Training Accuracy= 0.505
Iter 35000, Minibatch Loss= 0.6932, Training Accuracy= 0.500
Iter 35500, Minibatch Loss= 0.6932, Training Accuracy= 0.485
Iter 36000, Minibatch Loss= 0.6909, Training Accuracy= 0.590
Iter 36500, Minibatch Loss= 0.6936, Training Accuracy= 0.495
Iter 37000, Minibatch Loss= 0.6925, Training Accuracy= 0.520
Iter 37500, Minibatch Loss= 0.6917, Training Accuracy= 0.550
Iter 38000, Minibatch Lo

In [88]:
preds = np.round(preds)
corr = []
for a in range(len(test_dict)):
    fs = []
    for k in range(len(preds[a]) // log):
            fs.append(bit_to_freq(preds[a][k * log : (k+1) * log], N))
    corr.extend([fs[i] == test_dict[a][2][i] % N for i in range(len(fs))])
print(np.sum(corr) / (len(fs) * len(test_dict)))

0.34


(384, 2)